In [100]:
type Value =
| AInt of int
| ASeq of Value list
module Decoder =
    let isInt num = System.Text.RegularExpressions.Regex(@"\d").IsMatch(string num)
    let rec numDecoder (state:string) (tokens:char list):(string*char list) =
        match tokens with
        | [] -> "", tokens
        | token :: tail when (isInt token) ->
            numDecoder (state + string token) tail
        | _ -> state, tokens

    let rec decoder (state:Value list) (tokens:char list):(Value list*char list) =
        match tokens with
        | [] -> state, tokens
        | '[' :: tail -> 
            let aSeq, aTail = decoder List.empty tail
            let aState = [ Value.ASeq aSeq ] |> List.append state
            decoder aState aTail
        | ']' :: tail ->
            state, tail
        | ',' :: tail -> decoder state tail
        | num :: tail -> 
            let aNumber, aTail = numDecoder (string num) tail
            let value = aNumber |> int |> Value.AInt
            decoder ([value] |> List.append state) aTail

    let decode (line:string):Value =
        line
        |> List.ofSeq
        |> decoder List.empty
        |> fst
        |> List.head

module Encoder =
    let rec toString (value:Value) =
        match value with
        | Value.AInt i -> string i
        | Value.ASeq aSeq ->
            let content =
                aSeq
                |> List.map toString
                |> String.concat ","
            "[" + content + "]"

module Comparer =
    type Outcome =
    | Good
    | Bad
    | Tie
    let rec compare (a:Value) (b:Value) =
        let listCompare (la:Value list) (lb:Value list) =
            display $"{la.Length} - {lb.Length}"
            if la.Length < lb.Length then
                false
            else if lb.IsEmpty then
                false
            else
            Seq.zip la lb
            |> Seq.map (fun (va, vb) -> compare va vb)
            |> Seq.reduce (&&)

        match a, b with
        | Value.AInt ia, Value.AInt ib -> 
            display $"{ia} - {ib}"
            ia - ib < 1
        | Value.ASeq la, Value.ASeq lb ->
            listCompare la lb
        | Value.AInt _, Value.ASeq lb ->
            listCompare [a] lb
        | Value.ASeq la, Value.AInt _ ->
            listCompare la [b]

let ResolutionFolder = __SOURCE_DIRECTORY__
let lines = 
    File.ReadLines( ResolutionFolder + "/testcase13.txt") 
    |> List.ofSeq
    |> List.filter ( String.IsNullOrEmpty >> not )

let signal =
    lines
    |> Seq.chunkBySize 2
    |> Seq.map ( Seq.map Decoder.decode )
    |> Seq.map List.ofSeq

let parsed =
    signal
    |> Seq.map ( Seq.map Encoder.toString )
    |> Seq.concat

let original = 
    lines
    |> Seq.filter ( String.IsNullOrEmpty >> not )

Seq.zip original parsed
|> Seq.mapi (fun idx (a, b) -> (a = b), $"{idx + 1}  {a}  |  {b}" )
|> Seq.filter (fst >> not)
|> display

signal
|> Seq.map ( fun it -> Comparer.compare it.[0] it[1] )
|> Seq.mapi (fun idx it -> idx + 1 ,it)
//|> Seq.filter snd
//|> Seq.sumBy fst
|> display

[ "[[4,4],4,4]"; "[[4,4],4,4,4]" ]
|> List.map Decoder.decode
|> ( fun it -> Comparer.compare it.[0] it[1] )
|> sprintf "%A"


(empty)

5 - 5

1 - 1

1 - 1

3 - 5

...

8 - 8

9 - 9

index,Item1,Item2
0,1,True
1,2,True
2,3,True
3,4,True
4,5,True
5,6,True
6,7,False
7,8,False


3 - 4

true